In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

In [ ]:
import sys
from datawand.parametrization import ParamHelper
ph = ParamHelper('..', 'LinkPrediction', sys.argv)

# Parameters

In [ ]:
K = 30#ph.get("top_first_days")

In [ ]:
if K == None:
    timeframe = 86400*7
else:
    timeframe = 86400#

In [ ]:
models = ["online","batch","batch+online","pop","pop+time"]
prefixes = ["onmf","offmf","bomf","pop","time_pop"]

# Rankings

In [ ]:
#ranking_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/link_prediction/rankings/topk20_dim10_exkFalse/"
ranking_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/link_prediction/rankings/topk20_dim10_exkTrue/"

In [ ]:
rankings = [pd.read_csv("%s/%s_%s.csv" % (ranking_dir,pref,str(K))) for pref in prefixes]

In [ ]:
[len(df) for df in rankings]

In [ ]:
for i in range(len(rankings)):
    min_time = rankings[i]['time'].min()
    rankings[i]["timeframe"] = (rankings[i]['time']-min_time)//timeframe

# Results

### a.) average performance (online DCG)

The average performance for the offline batch model is confusing (it is only bad on the first day)

In [ ]:
def show_mean_dcg(with_first_day=True):
    if with_first_day:
        mean_dcgs = [df["dcg"].mean() for df in rankings]
    else:
        mean_dcgs = [df[df["time"]>(df["time"].min()+86400)]["dcg"].mean() for df in rankings]
    return pd.DataFrame(list(zip(models, mean_dcgs)), columns=["model","dcg"]).sort_values("dcg", ascending=False).reset_index(drop=True)

#### Global mean performance

In [ ]:
show_mean_dcg(True)

#### Mean performance without first day

In [ ]:
show_mean_dcg(False)

**Exclude known: False**
0 	online 	0.139660
1 	batch+online 	0.131745
2 	pop+time 	0.124183
3 	pop 	0.077110
4 	batch 	0.064587

**Exclude known: True - Miért teljesen uaz?**
0 	online 	0.139660
1 	batch+online 	0.131745
2 	pop+time 	0.124183
3 	pop 	0.077110
4 	batch 	0.064587

### b.) Performance over time

In [ ]:
for idx, ranking in enumerate(rankings):
    averages = ranking.groupby("timeframe")["dcg"].mean()
    plt.plot(averages, label=models[idx])
plt.legend()

### c.) Number of records over time

In [ ]:
cnt = rankings[0].groupby("timeframe")["dcg"].count()
plt.plot(cnt)